<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 5px; height: 70px">

## Notebook 1: Data Collection

### Goals for data collection
___
The aim for the CNN model is to be able to predict the appropriate keyboard input based on the information on the battle screen. Thus, the target variable for our problem is the keyboard input, while the predictor variable will be the game screen.

For this reason, we will need to collect the following information from our play sessions:
- Screenshots of the game screen
- Keyboard inputs when the screenshot is taken

We will also need a way to pair each image with its respective input, as images collected are saved as seperate files instead of simply being appended to a Python list.

### Imports

In [1]:
import cv2 as cv
from datetime import datetime
import time
import keyboard as kb
import pandas as pd
import os

from windowcapture import WindowCapture
import capstone_library as cl

#### Creating directories
We use today's date to create new directories for the images and the input dataframe. The date will also be used to name the images and for row indexing.

In [2]:
# Get today's date
today = datetime.now().strftime('%Y-%m-%d')

# Define paths to save images to
imgpath = ('../data/raw_img/{}/'.format(str(today)))
if not os.path.exists(imgpath):
    for mon in cl.opponents:
        os.makedirs(imgpath + mon + '/')

# Define path to save inputs to
inputpath =  ('../data/input/{}/'.format(str(today)))
if not os.path.exists(inputpath):
    os.makedirs(inputpath + '/')

### Initialising `WindowCapture` objects
___

The `WindowCapture` and `Vision` classes are imported from seperate Python scripts (`Vision` is imported into the `capstone_library.py` script, which is then imported into this notebook). Objects of these classes are used to perform the following tasks:

1. `WindowCapture`: Detect an open instance of the game window, then capture the battle scene in the window, allowing us to script a screenshot capture function.

    - Passing `portrait=True` when instantiating a `WindowCapture` object will capture the position of the enemy portrait instead, which we will use for computer vision capability.

2. `Vision`: Uses OpenCV template matching to detect matches to a given template when passed an input image. This is used in conjunction with the above, to identify the opponent in the battle scene.

Below is a visualisation of the images read from an open window of _Close Combat_. Image 1 is a capture of the battle scene while Image 2 captures the opponent portrait.

<img src="images/battle_screen.png"/>

The portraits used as detection templates are shown below, corresponding to `poliwrath`, `heracross` and `mienshao`, the three opponent Pokemon for which we will perform modelling.

<img src="../data/portrait/poliwrath.png"/>
<img src="../data/portrait/heracross.png"/>
<img src="../data/portrait/mienshao.png"/>

It may seem simpler to only capture one screenshot of the entire game screen, which would reduce the number of `WindowCapture` objects running simultaneously from 2 to 1. However, note that for the purposes of prediction, the bottom portrait section offers little to no information regarding the state of the battle scene. Cropping out the portraits will facilitate the model in focusing on meaningful information within the battle scene for performing predictions. It also has the benefit of reducing file size, which is helpful when mass collecting images.

While the `WindowCapture` objects are initialised here, the `Vision` objects are initialised during the data collection process, within the `cl.portraitcheck()` function. Refer to the data collection process section.

In [3]:
# Initialise WindowCapture. Make sure the game window is open first!

# wincap object captures battle scene(Image 1)
wincap = WindowCapture('pkmncc_09')

# portcap object captures enemy portrait(Image 2)
portcap = WindowCapture('pkmncc_09', portrait=True)

#### Troubleshooting `WindowCapture`
The commented code below can be run to troubleshoot the window capture capability. If the window capture is functioning, a window will open to show successive images captured in real-time (the images will appear to be a video stream, as images will consecutively replace each other).

In [4]:
# Troubleshooting window capture
# while(True):
#     screenshot = wincap.get_screenshot()
#     cv.imshow('test', screenshot)
#     if cv.waitKey(1) & 0xFF == ord('q'):
#         cv.destroyAllWindows()
#         break

### Data collection process
___
The data collection process can be summarised as follows:
1. While the `run_collect` function is running, repeatedly capture the battle scene and enemy portrait area.

2. Check the portrait area for a known enemy Pokemon's portrait. (Known portraits are saved in `..\data\portrait` and are used as templates for matching.) 
3. When a known portrait is detected, save the battle scene image to the specified directory.
    - Images are saved with a name in the datetime format (at the scale of milliseconds) so that they do not overwrite each other during data collection.
    - The `Vision` objects are used to detect the opponent Pokemon. Images are sorted into different folders based on the opponent Pokemon detected.
4. At the same time, the keyboard inputs, date/time of recording and opponent's name are appended to their respective lists. Each recording of keyboard inputs is saved as a list of 7 integers, which indicate whether the respective key is pressed during the recording.
    - Keys are recorded in this order: `['SPACEBAR', 'a', 's', 'd', 'j', 'k', 'l']`. For example, a input recording of `[0, 0, 0, 1, 0, 1, 0]` indicates that the `d` and `k` keys were pressed during the time of recording.
5. Inputs and images are recorded this way at an interval of 0.25 seconds.
6. When the user terminates the collection function, all lists are combined into a single dataframe, with each input given its own column. The dataframe is then exported to the specified directory. Each row of the dataframe can be matched to its respective image by comparing the `time` column and the name of the image.

We can then move on to EDA and data engineering.


In [5]:
'''Defining functions to collect inputs and screen captures'''

# collect() function captures the screenshots, keys and other information specified above
def collect(input_l, time_l, opp_n):
    timenow = today + ',' + datetime.now().strftime("%H-%M-%S-%f")[:-3]
    screenshot = wincap.get_screenshot()
    portshot = portcap.get_screenshot()

    # Only saves inputs and screenshots when a known enemy Pokemon portrait is detected
    # cl.portraitcheck() uses Vision objects for each Pokemon, for template matching
    mon = cl.portraitcheck(portshot)
    if mon:
        input_l.append(cl.keylog())
        time_l.append(timenow)
        opp_n.append(mon)
        cv.imwrite(imgpath + mon + '/{}.png'.format(timenow),
            screenshot)
    return timenow

# run_collect function loops thorugh a while loop to repeatedly capture the data, then saves the dataframe once completed.
def run_collect():
    # Initialise lists to collect inputs, time and opponent name
    input_list = []
    time_list = []
    opp_name = []
    print('Running collection...')
    # Collect data, break on q press
    while True:
        timenow = collect(input_list, time_list, opp_name)
        time.sleep(0.25)
        if kb.is_pressed('q'):
            print('Quitting...')
            break

    # Save inputs to csv
    df = pd.DataFrame({'time':time_list,
                        'opp':opp_name})
    df[cl.controls] = pd.DataFrame(input_list, index=df.index)
    filename ='{}.csv'.format(timenow)
    df.to_csv(inputpath + filename, index=False)
    print('{} has been saved to {}'.format(filename, inputpath))

In [6]:
'''
Run this function to collect screen captures, corresponding input and time.
Screencaps are sorted by opponent and saved to the respective folder.
Inputs are saved to a csv.
Press q to quit.
'''
df = run_collect()

Running collection...
Quitting...
2024-02-25,14-12-44-596.csv has been saved to ../data/input/2024-02-25/


In [ ]:
'''
Code in this block was used to capture the character portraits for character detection and does not need to be rerun
'''
# savepath = '../data/portrait/'

# # Initialise WindowCapture to catch portrait position
# portcap = WindowCapture('pkmncc_09',portrait=True)

# while(True):

#     # get an updated image of the portrait position
#     portshot = portcap.get_screenshot()
#     cv.imshow('portrait', portshot)

#     # press e to take screenshot of portrait
#     if kb.is_pressed('e'):
#         cv.imwrite(savepath + str(datetime.now().strftime("%Y%m%d-%H%M%S")) + '.png',
#         portshot)

#     if kb.is_pressed('q'):
#         cv.destroyAllWindows()
#         break